# Web scraping NBA transaction data from [basketball-reference.com](https://www.basketball-reference.com/)

In [1]:
# import bs4
from bs4 import BeautifulSoup
import requests
import csv
import re
import pandas as pd
# import ast

### HTML files

In [114]:
years = list(range(1991, 2025))

url_blank = "https://www.basketball-reference.com/leagues/NBA_{}_transactions.html"

In [196]:
# save HTML files for each year
# for year in [2015]:
for year in years:
    url = url_blank.format(year)
    response = requests.get(url)

    with open("./Data/transaction_htmls/transactions_{}.html".format(year), "w+") as f:
        f.write(response.text)

### Waived players data

In [199]:
# extract waived data from HTMLs and save as a CSV file
with open("./Data/waived_data.csv", "w+", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["season_start", "date", "player_name", "team_abv"])

    for year in years:
        with open("./Data/transaction_htmls/transactions_{}.html".format(year)) as h:
            page = h.read()

        soup = BeautifulSoup(page, "html.parser")

        page_index = soup.find("ul", attrs={"class":"page_index"})

        for entry in page_index.find_all("p"):
            if "waived" in entry.get_text():
                writer.writerow([year-1,
                                 entry.parent.find("span").get_text(),
                                 entry.find("a", attrs={"href":re.compile("/players/")}).get_text(),
                                 entry.find("a", attrs={"data-attr-from":True})["data-attr-from"]])

### Traded players data

In [173]:
# helper function for extracting each traded player (excluding draft picks) for a given entry and writing to the CSV file
def write_trades(entry, season_start, csv_writer):
    for tag in entry.find_all("a"):
        if "data-attr-from" in tag.attrs:
            current_team = tag["data-attr-from"]
        elif "data-attr-to" in tag.attrs:
            current_team = tag["data-attr-to"]
        elif ("href" in tag.attrs) and ("/players/" in tag["href"]) and ("was later selected" not in tag.next_sibling):
            csv_writer.writerow([season_start, entry.parent.find("span").get_text(), tag.get_text(), current_team])

In [201]:
# extract traded data from HTMLs and save as a CSV file
with open("./Data/traded_data.csv", "w+", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["season_start", "date", "player_name", "team_abv"])

    for year in years:
        with open("./Data/transaction_htmls/transactions_{}.html".format(year)) as h:
            page = h.read()

        soup = BeautifulSoup(page, "html.parser")

        page_index = soup.find("ul", attrs={"class":"page_index"})

        for entry in page_index.find_all("p"):
            if "traded" in entry.get_text():
                write_trades(entry, year-1, writer)

### Check for players labelled as coaches or executives

(Manually fill these in.)

In [176]:
for year in years:
    with open("./Data/transaction_htmls/transactions_{}.html".format(year)) as f:
        page = f.read()

    soup = BeautifulSoup(page, "html.parser")

    page_index = soup.find("ul", attrs={"class":"page_index"})
    entries = page_index.find_all("p")
    waived_entries = [entry for entry in entries if "waived" in entry.get_text()]
    traded_entries = [entry for entry in entries if "traded" in entry.get_text()]

    for entry in waived_entries:
        if len(entry.find_all("a", attrs={"href":re.compile("/coaches/")})) > 0:
            print(year, entry.find_all("a", attrs={"href":re.compile("/coaches/")}), "(waived)")
        if len(entry.find_all("a", attrs={"href":re.compile("/executives/")})) > 0:
            print(year, entry.find_all("a", attrs={"href":re.compile("/executives/")}), "(waived)")

    for entry in traded_entries:
        if len(entry.find_all("a", attrs={"href":re.compile("/coaches/")})) > 0:
            print(year, entry.find_all("a", attrs={"href":re.compile("/coaches/")}), "(traded)")
        if len(entry.find_all("a", attrs={"href":re.compile("/executives/")})) > 0:
            print(year, entry.find_all("a", attrs={"href":re.compile("/executives/")}), "(traded)")


1992 [<a href="/coaches/dunlemi01c.html">Mike Dunleavy</a>] (traded)
1996 [<a href="/coaches/rileypa01c.html">Pat Riley</a>] (traded)
1997 [<a href="/executives/wallach99x.html">Chris Wallace</a>] (traded)
2014 [<a href="/coaches/kiddja01c.html">Jason Kidd</a>] (traded)
2018 [<a href="/executives/perrysc99x.html">Scott Perry</a>] (traded)


In [181]:
with open("./Data/traded_data.csv", "a", newline="") as f:
    writer = csv.writer(f)

    writer.writerow([1991, "May 11, 1992", "Mike Dunleavy", "LAL"])
    writer.writerow([1995, "September 1, 1995", "Pat Riley", "NYK"])
    writer.writerow([1996, "May 27, 1997", "Chris Wallace", "BOS"])
    writer.writerow([2013, "June 30, 2014", "Jason Kidd", "BRK"])
    writer.writerow([2017, "July 14, 2017", "Scott Perry", "SAC"])

### Concatenating waived and traded data

In [217]:
# concatenate data using DataFrames
waived_df = pd.read_csv("./Data/waived_data.csv")
waived_df["waived"] = 1
waived_df["traded"] = 0

traded_df = pd.read_csv("./Data/traded_data.csv")
traded_df["waived"] = 0
traded_df["traded"] = 1

transaction_df = pd.concat([waived_df, traded_df], ignore_index=True)

In [ ]:
# decoding player names with special characters
transaction_df["player_name"] = transaction_df["player_name"].str.encode('latin-1').str.decode('utf-8')

In [218]:
transaction_df.sample(10)

,season_start,date,player_name,team_abv,waived,traded
7940,2019,"July 6, 2019",Anthony Davis,NOP,0,1
822,1999,"October 20, 1999",Maceo Baston,CHI,1,0
4429,2022,"October 17, 2022",Kemba Walker,DET,1,0
3339,2018,"December 2, 2018",Billy Preston,CLE,1,0
3746,2020,"December 18, 2020",Trevon Bluiett,UTA,1,0
8152,2020,"March 18, 2021",Torrey Craig,MIL,0,1
8237,2021,"August 6, 2021",Chandler Hutchison,WAS,0,1
1170,2003,"October 22, 2003",Jeryl Sasser,ORL,1,0
5063,1994,"February 14, 1995",Marcelo Nicola,HOU,0,1
5622,2000,"September 25, 2000",Popeye Jones,DEN,0,1


In [227]:
# save transaction data as a CSV file
transaction_df.to_csv("./Data/transaction_data.csv", index=False)

### Adding player IDs

In [2]:
%pip install nba_api
%pip install unidecode

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from nba_api.stats.static import players
from unidecode import unidecode

In [4]:
# list of dictionaries, one for each player, containing id, name, etc.
all_players = players.get_players()

In [6]:
transaction_df = pd.read_csv("Data/transaction_data.csv")

In [9]:
transaction_df

,season_start,date,player_name,team_abv,waived,traded
0,1990,"July 5, 1990",Anthony Bowie,HOU,1,0
1,1990,"July 9, 1990",Randy Allen,SAC,1,0
2,1990,"July 9, 1990",Michael Jackson,SAC,1,0
3,1990,"July 24, 1990",Melvin Turpin,WSB,1,0
4,1990,"August 10, 1990",Scott Haffner,MIA,1,0
...,...,...,...,...,...,...
8674,1991,"May 11, 1992",Mike Dunleavy,LAL,0,1
8675,1995,"September 1, 1995",Pat Riley,NYK,0,1
8676,1996,"May 27, 1997",Chris Wallace,BOS,0,1
8677,2013,"June 30, 2014",Jason Kidd,BRK,0,1


In [15]:
no_match_count = 0
many_match_count = 0

for i in range(len(transaction_df)):
    num_matches = len(players.find_players_by_full_name(unidecode(transaction_df.loc[i, "player_name"])))
    if num_matches == 0:
        no_match_count += 1
    elif len(players.find_players_by_full_name(unidecode(transaction_df.loc[i, "player_name"]))) > 1:
        many_match_count += 1

print(no_match_count)
print(many_match_count)

1005
159


In [16]:
players.find_players_by_full_name("Turpin")

[{'id': 78386,
  'full_name': 'Mel Turpin',
  'first_name': 'Mel',
  'last_name': 'Turpin',
  'is_active': False}]

In [18]:
no_id_df = transaction_df[transaction_df["player_name"].apply(unidecode).apply(players.find_players_by_full_name).apply(len) == 0]

In [21]:
many_ids_df = transaction_df[transaction_df["player_name"].apply(unidecode).apply(players.find_players_by_full_name).apply(len) > 1]

In [19]:
no_id_df

,season_start,date,player_name,team_abv,waived,traded
3,1990,"July 24, 1990",Melvin Turpin,WSB,1,0
14,1990,"October 19, 1990",Steve Bardo,ATL,1,0
28,1990,"October 30, 1990",Mike Holton,CHH,1,0
81,1990,"May 21, 1991",Dave Greenwood,SAS,1,0
134,1991,"November 12, 1991",Steve Bardo,SAS,1,0
...,...,...,...,...,...,...
8664,2023,"February 8, 2024",Dimitrios Agravanis,SAC,0,1
8665,2023,"February 8, 2024",Ismael Kamagate,DEN,0,1
8669,2023,"February 8, 2024",Vasilije Micić,OKC,0,1
8676,1996,"May 27, 1997",Chris Wallace,BOS,0,1


In [22]:
many_ids_df

,season_start,date,player_name,team_abv,waived,traded
64,1990,"December 24, 1990",Reggie Williams,SAS,1,0
90,1991,"October 2, 1991",Larry Drew,LAL,1,0
105,1991,"October 28, 1991",Gerald Henderson,HOU,1,0
122,1991,"October 30, 1991",Michael Smith,BOS,1,0
172,1991,"January 7, 1992",Gerald Henderson,HOU,1,0
...,...,...,...,...,...,...
7560,2016,"January 7, 2017",Mike Dunleavy,CLE,0,1
8017,2019,"February 6, 2020",Glenn Robinson,GSW,0,1
8131,2020,"December 2, 2020",John Wall,WAS,0,1
8470,2022,"February 9, 2023",John Wall,LAC,0,1


In [27]:
players.find_players_by_full_name("Vasilije Micić")

[{'id': 203995,
  'full_name': 'Vasilije Micić',
  'first_name': 'Vasilije',
  'last_name': 'Micić',
  'is_active': True}]

In [26]:
unidecode("Micić")

'Micic'

In [39]:
no_match_count = 0
unique_match_count = 0
many_match_count = 0

for i in no_id_df.index:
    num_matches = len(players.find_players_by_last_name((no_id_df.loc[i, "player_name"]).split()[-1]))
    if num_matches == 0:
        no_match_count += 1
    elif num_matches == 1:
        unique_match_count += 1
    else:
        many_match_count += 1

print("No match count:", no_match_count)
print("Unique match count:", unique_match_count)
print("Many match count:", many_match_count)

No match count: 456
Unique match count: 147
Many match count: 402


In [31]:
players.find_players_by_full_name("Holton")

[{'id': 77047,
  'full_name': 'Michael Holton',
  'first_name': 'Michael',
  'last_name': 'Holton',
  'is_active': False}]

In [41]:
match_count = 0

for i in no_id_df.index:
    num_matches = len(players.find_players_by_full_name((no_id_df.loc[i, "player_name"])))
    if num_matches > 0:
        match_count += 1

print(match_count)

25
